In [1]:
import gzip
import math
import random
from collections import defaultdict
import json

In [2]:
dataDir = "/Users/Judy-Ccino412/Desktop/cse158/data/"

In [3]:
path = dataDir + "goodreads_reviews_comics_graphic.json.gz"
f = gzip.open(path, 'rt', encoding="utf8")

In [4]:
data = []
for l in f:
    d = json.loads(l)
    data.append(d)
    
f.close()

### Q1

In [5]:
data[0]['user_id'], data[0]['book_id'], data[0]['review_id'], data[0]['rating']

('dc3763cdb9b2cae805882878eebb6a32',
 '18471619',
 '66b2ba840f9bd36d6d27f46136fe4772',
 3)

In [6]:
# Extract a few utility data structures
usersPerItem = defaultdict(set) # Maps an item to the users who rated it
itemsPerUser = defaultdict(set) # Maps a user to the items that they rated
reviewId = {}
ratingDict = {} # To retrieve a rating for a specific user/item pair

for d in data:
    user,item = d['user_id'], d['book_id']
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    ratingDict[(user,item)] = d['rating']
    reviewId[item] = d['review_id']

In [7]:
# Extract per-user and per-item averages (useful later for rating prediction)
userAverages = {}
itemAverages = {}

for u in itemsPerUser:
    rs = [ratingDict[(u,i)] for i in itemsPerUser[u]]
    userAverages[u] = sum(rs) / len(rs)
    
for i in usersPerItem:
    rs = [ratingDict[(u,i)] for u in usersPerItem[i]]
    itemAverages[i] = sum(rs) / len(rs)

In [8]:
# Jaccard Similarity
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [9]:
def mostSimilarItem(i, N):
    similarities = []
    users = usersPerItem[i]
    for i2 in usersPerItem:
        if i2 == i: 
            continue
        sim = Jaccard(users, usersPerItem[i2])  # ie. sim = common users / union users
        similarities.append((sim,i2))
    similarities.sort(key=lambda tup: (-tup[0], tup[1]))
    return similarities[:N]


In [10]:
query = data[0]['book_id']
query

'18471619'

In [11]:
# Q1 answer
mostSimilarItem(query, 10) 

[(0.16666666666666666, '25334626'),
 (0.14285714285714285, '25659811'),
 (0.13793103448275862, '18369278'),
 (0.13157894736842105, '18430205'),
 (0.12903225806451613, '20299669'),
 (0.125, '17995154'),
 (0.12121212121212122, '18853527'),
 (0.12121212121212122, '23093378'),
 (0.12121212121212122, '23241671'),
 (0.11764705882352941, '18734070')]

### Q2

#### (a) Choosing the N items most similar to the user’s favorite (i.e., highest rated) item

In [12]:
u = data[0]['user_id']
u

'dc3763cdb9b2cae805882878eebb6a32'

In [13]:
# All items that User u has
i_of_u = itemsPerUser[u]
i_of_u

{'18471619'}

In [14]:
# find favorite item of User u
def fav_i_of_user(u, all_items_of_u):
    ratings = [-1]
    for i in sorted(all_items_of_u):
        i_rating = ratingDict[(u, i)]
        if i_rating > max(ratings):
            ratings.append(i_rating)
            fav_i = i
    return fav_i

In [15]:
fav_i_of_u = fav_i_of_user(u, i_of_u)
fav_i_of_u

'18471619'

In [16]:
# Q2(a) answer
# 10 items most similar to the User u’s favorite
mostSimilarItem(fav_i_of_u, 10) 

[(0.16666666666666666, '25334626'),
 (0.14285714285714285, '25659811'),
 (0.13793103448275862, '18369278'),
 (0.13157894736842105, '18430205'),
 (0.12903225806451613, '20299669'),
 (0.125, '17995154'),
 (0.12121212121212122, '18853527'),
 (0.12121212121212122, '23093378'),
 (0.12121212121212122, '23241671'),
 (0.11764705882352941, '18734070')]

#### (b) Finding the N most similar users, and recommending each of their their favorite (highest rated) items.

In [17]:
def mostSimilarUser(u, N):
    similarities = []
    items = itemsPerUser[u]
    candidateUsers = set()
    for i in items:
        candidateUsers = candidateUsers.union(usersPerItem[i])
    for u2 in candidateUsers:
        if u2 == u: 
            continue
        sim = Jaccard(items, itemsPerUser[u2])  # ie. sim = common items / union items
        similarities.append((sim,u2))
    similarities.sort(key=lambda tup: (-tup[0], tup[1]))
    return similarities[:N]

In [18]:
# 10 most similar users to User u with the scores (doesn't skip users)
top_ten = mostSimilarUser(u, 10)
top_ten

[(1.0, '4ae069d704b11bdf12c25fe640f75ff0'),
 (0.3333333333333333, '6470c7f5e3468ba34e9fe628960fbbf1'),
 (0.25, '6497ca91df3c182006874c96a8530b37'),
 (0.2, '033cf640dfa6f85eb146c39787289628'),
 (0.14285714285714285, '5510684ab6c18f2dd493787e66b2722c'),
 (0.05555555555555555, '17f73ea38e97307935c0d3b6ca987b53'),
 (0.030303030303030304, 'a39b4249d201ef5ce5ea553bdd013e66'),
 (0.023809523809523808, '42519f961f79b61701bda60787b031cf'),
 (0.02040816326530612, '65a7975989734fc6e18b7d2bd2bcb49f'),
 (0.014925373134328358, '0fafb6f0843124383f4e2c5a2090fb09')]

In [19]:
# get the 10 most similar users to User u (doesn't skip users)
ten_similar_users = [tup[1] for tup in mostSimilarUser(u, 10)]
ten_similar_users

['4ae069d704b11bdf12c25fe640f75ff0',
 '6470c7f5e3468ba34e9fe628960fbbf1',
 '6497ca91df3c182006874c96a8530b37',
 '033cf640dfa6f85eb146c39787289628',
 '5510684ab6c18f2dd493787e66b2722c',
 '17f73ea38e97307935c0d3b6ca987b53',
 'a39b4249d201ef5ce5ea553bdd013e66',
 '42519f961f79b61701bda60787b031cf',
 '65a7975989734fc6e18b7d2bd2bcb49f',
 '0fafb6f0843124383f4e2c5a2090fb09']

In [20]:
# get all items of the similar uses
all_items_of_sus = {}
for su in ten_similar_users:
    all_items_of_su = sorted(itemsPerUser[su])
    all_items_of_sus[su] = (all_items_of_su)

#all_items_of_sus

Strategy: Skip the current similar user ONLY WHEN all the items of this similar user are purchased by User u, OTHERWISE, move to the NEXT FAVORITE ITEM of this similar user instead of skipping this similar user.

In [21]:
# recommend an item for User u based on the fav item of a similar user
def recommend_for_u_with_su(su, all_items_of_su, i_of_u):
    if len(all_items_of_su) == 0:
        #print('no item satisfied: ' + su)
        return
    
    # get the favorite item of the current similar user su
    fav_i_of_su = fav_i_of_user(su, all_items_of_su)
    #print(fav_i_of_su)
    
    # if this item is not purchased by User u, recommend it
    if fav_i_of_su not in i_of_u:
        #print(fav_i_of_su)
        recommend_i = fav_i_of_su
        return recommend_i  
    
    # if this item is purchased by User u, recommend the next favorite item of su
    all_items_of_su = all_items_of_su[1:] 
    recommend_i = recommend_for_u_with_su(su, all_items_of_su, i_of_u)
    

In [22]:
# recommend for User u 
recommend_items = []
scores = []
for su_i in range(len(ten_similar_users)):
    su = ten_similar_users[su_i]
    #print(su)
    all_items_of_su = sorted(all_items_of_sus[su])
    recommend_i = recommend_for_u_with_su(su, all_items_of_su, i_of_u)
    
    # if the similar user still has items that User u hasn't purchase, continue the recommendations
    if recommend_i is not None:
        recommend_items.append(recommend_i)
        score = [i[0] for i in top_ten if i[1] == su]
        scores.append(score)
        all_items_of_sus[su].remove(recommend_i)
        all_items_of_sus.update({su : all_items_of_sus[su]})
        
    # if the item is purchased by User u, SKIP THIS ITEM and
    # move to the next similar user ONLY IF all the items of the similar user is purchase by User u
    else:
        su_next = ten_similar_users[su_i + 1]
        all_items_of_su_next = sorted(all_items_of_sus[su_next])
        recommend_i = recommend_for_u_with_su(su_next, all_items_of_su_next, i_of_u)
        recommend_items.append(recommend_i)
        score = [i[0] for i in top_ten if i[1] == su_next]
        scores.append(score)
        all_items_of_sus[su_next].remove(recommend_i)
        all_items_of_sus.update({su_next : all_items_of_sus[su_next]})
        
    # recommend 10 items
    if len(recommend_items) == 10:
        break
        
recommend_items

['10767466',
 '5805',
 '17570797',
 '15704307',
 '10138607',
 '12434747',
 '17995248',
 '10105459',
 '10997645',
 '10361139']

In [23]:
score = [score[0] for score in scores]

In [24]:
# Q2(b) answer 
[(s, i) for s,i in zip(score, recommend_items)]

[(0.3333333333333333, '10767466'),
 (0.3333333333333333, '5805'),
 (0.25, '17570797'),
 (0.2, '15704307'),
 (0.14285714285714285, '10138607'),
 (0.05555555555555555, '12434747'),
 (0.030303030303030304, '17995248'),
 (0.023809523809523808, '10105459'),
 (0.02040816326530612, '10997645'),
 (0.014925373134328358, '10361139')]

### Q3

In [25]:
i1 = data[0]['book_id']
i1

'18471619'

#### (a) Pearson similarity with denominator only in terms of shared items (i.e., Ui ∩ Uj ) in the denominator

In [26]:
def Pearson1(i1, i2):
    # Between two items
    iBar1 = itemAverages[i1]    # avg rating of i1
    iBar2 = itemAverages[i2]    # avg rating of i2
    # intersection of users that have rated items i and j
    inter = usersPerItem[i1].intersection(usersPerItem[i2])
    numer = 0
    denom1 = 0
    denom2 = 0
    
    # numerator is the intersection of users
    for u in inter:
        numer += (ratingDict[(u,i1)] - iBar1)*(ratingDict[(u,i2)] - iBar2)
        
    # denominator in terms of shared items
    for u in inter: 
        denom1 += (ratingDict[(u,i1)] - iBar1)**2
        denom2 += (ratingDict[(u,i2)] - iBar2)**2
    denom = math.sqrt(denom1) * math.sqrt(denom2)
    if denom == 0: return 0
    return numer / denom

In [27]:
def mostSimilarItem_by_pearson1(i, N):
    similarities = []
    users = usersPerItem[i]
    for i2 in usersPerItem:
        if i2 == i: 
            continue
        sim = Pearson1(i, i2) # Could use alternate similarity metrics straightforwardly
        similarities.append((sim,i2))
    similarities.sort(key=lambda tup: (-tup[0], tup[1]))
    return similarities[:N]

In [28]:
# Q3(a) answer
mostSimilarItem_by_pearson1(i1, 10)

[(1.0000000000000002, '1103951'),
 (1.0000000000000002, '11986350'),
 (1.0000000000000002, '16007365'),
 (1.0000000000000002, '17132269'),
 (1.0000000000000002, '17571519'),
 (1.0000000000000002, '18468852'),
 (1.0000000000000002, '18527488'),
 (1.0000000000000002, '18594657'),
 (1.0000000000000002, '18624024'),
 (1.0000000000000002, '1882498')]

#### (b) Pearson similarity with denominator in terms of all items each user consumed (i.e., Ui or Uj for each term in the denominator

In [29]:
def Pearson2(i1, i2):
    # Between two items
    iBar1 = itemAverages[i1]
    iBar2 = itemAverages[i2]
    # intersection of users that have rated items i and j
    inter = usersPerItem[i1].intersection(usersPerItem[i2])
    numer = 0
    denom1 = 0
    denom2 = 0
    
    # numerator is the intersection of users
    for u in inter:
        numer += (ratingDict[(u,i1)] - iBar1)*(ratingDict[(u,i2)] - iBar2)
        
    # denominator in terms of all items each user consumed
    for u in usersPerItem[i1]:
        denom1 += (ratingDict[(u,i1)] - iBar1)**2
    for u in usersPerItem[i2]:
        denom2 += (ratingDict[(u,i2)] - iBar2)**2
    denom = math.sqrt(denom1) * math.sqrt(denom2)
    if denom == 0: return 0
    return numer / denom

In [30]:
def mostSimilarItem_by_pearson2(i, N):
    similarities = []
    users = usersPerItem[i]
    for i2 in usersPerItem:
        if i2 == i: 
            continue
        sim = Pearson2(i, i2) # Could use alternate similarity metrics straightforwardly
        similarities.append((sim,i2))
    similarities.sort(key=lambda tup: (-tup[0], tup[1]))
    return similarities[:N]

In [31]:
# Q3(b) answer
mostSimilarItem_by_pearson2(i1, 10)

[(0.31898549007874194, '20300526'),
 (0.18785865431369264, '13280885'),
 (0.17896391275176457, '18208501'),
 (0.16269036695641687, '21521612'),
 (0.16269036695641687, '25430791'),
 (0.1555075595594449, '1341758'),
 (0.1526351566298752, '6314737'),
 (0.15204888048160353, '4009034'),
 (0.1494406444160154, '988744'),
 (0.14632419481281994, '18430205')]

### Q4

In [32]:
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)

In [33]:
for d in data:
    user,item = d['user_id'], d['book_id']
    reviewsPerUser[user].append(d)
    reviewsPerItem[item].append(d)

In [34]:
ratingMean = sum([d['rating'] for d in data]) / len(data)
ratingMean

3.778138356523054

In [35]:
def predictRating(user,item):
    ratings = []
    similarities = []
    for d in reviewsPerUser[user]:
        i2 = d['book_id']
        if i2 == item: continue
        ratings.append(d['rating'] - itemAverages[i2])
        similarities.append(Jaccard(usersPerItem[item],usersPerItem[i2]))
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return itemAverages[item] + sum(weightedRatings) / sum(similarities)
    else:
        # User hasn't rated any similar items
        return ratingMean

In [36]:
def MSE_of_rating_prediction(predict, rating):
    SE = [(a-b)**2 for a,b in zip(predict, rating)]
    return sum(SE) / len(predict)

Strategy: Since it takes too long to run on the whole dataset, I instead make predictions on a random subset of size 10,000 without the loss of generality.

In [37]:
# extract subset of data with size 10000
sample = random.sample(data, 10000)

In [38]:
# make predictions on the sample
predictions = [predictRating(d['user_id'], d['book_id']) for d in sample]

In [39]:
ratings = [d['rating'] for d in sample]

In [40]:
# Q4 answer
# compute MSE on sample
MSE_of_rating_prediction(predictions, ratings)

0.8363993189608434

### Q5

#### (a) Modify the similarity function from Question 4 to use the cosine similarity

In [41]:
def Cosine(i1, i2):
    # Between two items
    inter = usersPerItem[i1].intersection(usersPerItem[i2])
    numer = 0
    denom1 = 0
    denom2 = 0
    for u in inter:
        numer += ratingDict[(u,i1)]*ratingDict[(u,i2)]
    for u in usersPerItem[i1]:
        denom1 += ratingDict[(u,i1)]**2
    for u in usersPerItem[i2]:
        denom2 += ratingDict[(u,i2)]**2
    denom = math.sqrt(denom1) * math.sqrt(denom2)
    if denom == 0: return 0
    return numer / denom

In [42]:
def predictRatingWithCos(user,item):
    ratings = []
    similarities = []
    for d in reviewsPerUser[user]:
        i2 = d['book_id']
        if i2 == item: 
            continue
        ratings.append(d['rating'] - itemAverages[i2])
        similarities.append(Cosine(item,i2))
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return itemAverages[item] + sum(weightedRatings) / sum(similarities)
    else:
        # User hasn't rated any similar items
        return ratingMean

In [43]:
# make predictions on the sample
predictions = [predictRatingWithCos(d['user_id'], d['book_id']) for d in sample]

In [44]:
# Q5(a) answer
# compute MSE on sample
MSE_of_rating_prediction(predictions, ratings)

0.8356020751105451

#### (b) Modify the similarity function from Question 4 to use the two definitions of Pearson similarity from Question 3

In [45]:
def predictRatingWithPearson1(user,item):
    ratings = []
    similarities = []
    for d in reviewsPerUser[user]:
        i2 = d['book_id']
        if i2 == item: 
            continue
        ratings.append(d['rating'] - itemAverages[i2])
        similarities.append(Pearson1(item,i2))
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return itemAverages[item] + sum(weightedRatings) / sum(similarities)
    else:
        # User hasn't rated any similar items
        return ratingMean

In [46]:
# make predictions on the sample
predictions = [predictRatingWithPearson1(d['user_id'], d['book_id']) for d in sample]

In [47]:
# Q5(b)-1 answer
# compute MSE on sample
MSE_of_rating_prediction(predictions, ratings)

4.406678744237735e+24

In [48]:
def predictRatingWithPearson2(user,item):
    ratings = []
    similarities = []
    for d in reviewsPerUser[user]:
        i2 = d['book_id']
        if i2 == item: 
            continue
        ratings.append(d['rating'] - itemAverages[i2])
        similarities.append(Pearson2(item,i2))
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return itemAverages[item] + sum(weightedRatings) / sum(similarities)
    else:
        # User hasn't rated any similar items
        return ratingMean

In [49]:
# make predictions on the sample
predictions = [predictRatingWithPearson2(d['user_id'], d['book_id']) for d in sample]

In [50]:
# Q5(b)-2 answer
# compute MSE on sample
MSE_of_rating_prediction(predictions, ratings)

1615.1700734159913

#### (c) Modify the similarity function from Question 4 to use the  Jaccard similarity, but interchanging users and items (i.e., in terms of the similarity between users Sim(u, v) rather than Sim(i, j)).

In [51]:
def predictRatingSimUser(user,item):
    ratings = []
    similarities = []
    for d in reviewsPerUser[user]:
        u2 = d['user_id']
        if u2 == user:
            continue
        ratings.append(d['rating'] - userAverages[u2])
        # in terms of the similarity between users Sim(u, v)
        similarities.append(Jaccard(itemsPerUser[user],itemsPerUser[u2]))
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return itemAverages[item] + sum(weightedRatings) / sum(similarities)
    else:
        # User hasn't rated any similar items
        return ratingMean

In [52]:
# make predictions on the sample
predictions = [predictRatingSimUser(d['user_id'], d['book_id']) for d in sample]

In [53]:
# Q5(c) answer
# compute MSE on sample
MSE_of_rating_prediction(predictions, ratings)

1.3361374533772217